In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
%matplotlib inline
import nltk
# for accuracy mertic
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
df=pd.read_csv('train.csv')
df.head(10)

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Complaint-Status,Consumer-disputes,Consumer-complaint-summary
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Closed with explanation,Yes,"Seterus, Inc a déposé un faux rapport auprès d..."
1,Tr-2,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,Closed with non-monetary relief,No,XX / XX / XXXX La requête en faillite n ° XXXX...
2,Tr-3,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,Closed with explanation,No,"El XXXX / XXXX / 15, estaba preparando el vuel..."
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,Closed with explanation,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea..."
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,Closed with explanation,No,J'ai obtenu un compte de crédit de soins pour ...
5,Tr-6,8/2/2016,Mortgage,"Loan modification,collection,foreclosure",NaN,8/3/2016,Closed with explanation,Yes,The owner of my original mortgage filed for ba...
6,Tr-7,3/26/2017,Credit reporting,Incorrect information on credit report,NaN,3/26/2017,Closed with explanation,No,J'ai été victime d'une fraude d'identité et j'...
7,Tr-8,10/15/2016,Bank account or service,Problems caused by my funds being low,Company has responded to the consumer and the ...,10/15/2016,Closed with explanation,No,"Je suis en train de faire faillite et, par con..."
8,Tr-9,1/18/2016,Debt collection,Cont'd attempts collect debt not owed,NaN,1/18/2016,Closed with explanation,Yes,Una agencia de cobranza me hizo adulterar de q...
9,Tr-10,8/17/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,8/17/2015,Closed with non-monetary relief,No,"Le XXXX / XXXX / 2015, j'ai reçu une réponse d..."


In [4]:
df.drop(columns=['Date-received', 'Date-sent-to-company','Complaint-ID'], axis=1, inplace=True)

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prasads\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prasads\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prasads\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
import re
from string import punctuation
from nltk.corpus import stopwords
stop_list = stopwords.words('english')

In [7]:
from nltk.stem.snowball import SnowballStemmer
s1= SnowballStemmer("english")
s2= SnowballStemmer("french")
s3= SnowballStemmer("german")
s4= SnowballStemmer("portuguese")
s5= SnowballStemmer("russian")
s6= SnowballStemmer("spanish")
s7= SnowballStemmer("swedish")

In [8]:
#text=df['Consumer-complaint-summary']
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = nltk.word_tokenize(text)
    tokens = [' '.join(w for w in tokens if w.lower() not in (stop_list + list(punctuation)))]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stemmer=[s1,s2,s3,s4,s5,s6,s7]
    for i in np.arange(0,7,1):
      stems = [stemmer[i].stem(t) for t in filtered_tokens]
    return " ".join(map(str, stems))

In [9]:
# tokenizedArticles = pd.DataFrame()
df['cleaned_tokens'] = df['Consumer-complaint-summary'].apply(lambda x: tokenize_and_stem(x))
df['word_count'] = df['cleaned_tokens'].apply(lambda x: len(x))

In [13]:
df.drop(columns=['Consumer-complaint-summary','word_count', 'Company-response', 'Consumer-disputes'], axis=1, inplace=True)

In [15]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['Complaint-reason'] = le.fit_transform(df['Complaint-reason'])
df['Transaction-Type'] = le.fit_transform(df['Transaction-Type'])

In [16]:
y=df['Complaint-Status']

In [17]:
df.drop(columns=['Complaint-Status'], axis=1, inplace=True)

In [18]:
df.head()

,Transaction-Type,Complaint-reason,cleaned_tokens
0,10,78,seterus inc déposé un faux rapport auprès des ...
1,5,71,xx xx xxxx la requête en faillite n ° xxxx du ...
2,0,145,el xxxx xxxx 15 estaba preparando el vuelo de ...
3,7,36,loan paid xxxx xxxx xxxx 4 years moved va move...
4,3,100,j'ai obtenu un compte de crédit de soins pour ...


In [19]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.5, max_features=200000,
                                 min_df=0.1, stop_words='english',
                                 use_idf=True, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(df['cleaned_tokens'])


print(tfidf_matrix.shape)

Wall time: 51 s
(43266, 59)


In [20]:
tfidf_df = pd.DataFrame(tfidf_matrix.A, columns=tfidf_vectorizer.get_feature_names(),index=df.index)
tfidf_df.head(15)

,00,account,ai,asked,bank,called,card,company,credit,credit report,...,time,told,una,une,xx,xx xx,xx xx xxxx,xx xxxx,xxxx xxxx xxxx,years
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.213506,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.093296,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.651814,0.330917,0.363020,0.361756,0.270891,0.000000
2,0.055973,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.124335,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.050641,0.211397,0.000000,0.072526,0.000000,0.000000,0.000000,0.064414,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.117935,0.059874,0.065682,0.065454,0.343092,0.372540
4,0.000000,0.000000,0.597075,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.112280,0.000000,0.000000,0.000000,0.000000,0.037861,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.153109,0.000000,0.154676,0.000000,0.000000,...,0.150971,0.149939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.357831
6,0.000000,0.000000,0.126968,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.043773,0.710325,0.252436,0.276925,0.512499,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.220066,0.095724,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.216005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.263136,0.000000,0.000000,0.000000,0.000000,0.000000,0.127388,0.000000
9,0.000000,0.000000,0.339365,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.350997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
df.drop(columns=['cleaned_tokens'], axis=1, inplace=True)

In [22]:
df.head()

,Transaction-Type,Complaint-reason
0,10,78
1,5,71
2,0,145
3,7,36
4,3,100


In [23]:
df.index

RangeIndex(start=0, stop=43266, step=1)

In [24]:
df_2=df.join(tfidf_df)
df_2.head()

,Transaction-Type,Complaint-reason,00,account,ai,asked,bank,called,card,company,...,time,told,una,une,xx,xx xx,xx xx xxxx,xx xxxx,xxxx xxxx xxxx,years
0,10,78,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.213506,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,5,71,0.093296,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.651814,0.330917,0.363020,0.361756,0.270891,0.00000
2,0,145,0.055973,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.124335,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,7,36,0.050641,0.211397,0.000000,0.072526,0.0,0.0,0.0,0.064414,...,0.0,0.0,0.000000,0.000000,0.117935,0.059874,0.065682,0.065454,0.343092,0.37254
4,3,100,0.000000,0.000000,0.597075,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.112280,0.000000,0.000000,0.000000,0.000000,0.037861,0.00000


In [25]:
X=df_2

In [27]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import Counter
from sklearn.datasets import make_classification

In [29]:
smt = SMOTE(random_state=42, k_neighbors=1)
X_SMOTE, y_SMOTE = smt.fit_sample(X, y) 

In [31]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(ratio='auto')
X_smt, y_smt = smt.fit_sample(X, y)

In [0]:
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(class_weight='balanced', multi_class='multinomial')
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_test)
print(metrics.classification_report(y_test, pred))

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_SMOTE, y_SMOTE, test_size=0.33, random_state=42)

In [33]:
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier(random_state=0, n_jobs=-1, class_weight='balanced')   # {class_label: weight}
model = clf1.fit(X_train, y_train)
predict = model.predict(X_test)
proba = model.predict_proba(X_test)
cm = metrics.confusion_matrix(y_test, predict)
print(metrics.classification_report(y_test, predict))

f:\envs\env1\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


                                 precision    recall  f1-score   support

                         Closed       0.98      0.99      0.98     11205
        Closed with explanation       0.85      0.83      0.84     11374
    Closed with monetary relief       0.93      0.96      0.95     11354
Closed with non-monetary relief       0.90      0.89      0.89     11331
              Untimely response       1.00      0.99      1.00     11331

                      micro avg       0.93      0.93      0.93     56595
                      macro avg       0.93      0.93      0.93     56595
                   weighted avg       0.93      0.93      0.93     56595



In [107]:
X_train

array([[5.00000000e+00, 4.30000000e+01, 4.00000000e+00, ...,
        0.00000000e+00, 2.48111542e-02, 0.00000000e+00],
       [1.00000000e+01, 3.40000000e+01, 3.74986875e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.00000000e+00, 2.40000000e+01, 4.00000000e+00, ...,
        0.00000000e+00, 6.99948693e-03, 0.00000000e+00],
       ...,
       [6.00000000e+00, 5.30000000e+01, 7.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.00000000e+00, 3.70000000e+01, 9.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.00000000e+00, 3.10000000e+01, 9.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [34]:
print( metrics.accuracy_score(y_test,predict))

0.9313190211149395


In [35]:
cm

array([[11058,   130,     5,    12,     0],
       [  162,  9441,   654,  1101,    16],
       [    6,   394, 10922,    31,     1],
       [   51,  1108,   131, 10036,     5],
       [    4,    73,     0,     3, 11251]], dtype=int64)

# Testing starts here

In [36]:
df1 = pd.read_csv('test.csv')
df1.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes,Consumer-complaint-summary
0,Te-1,8/18/2016,Bank account or service,"Account opening, closing, or management",Company has responded to the consumer and the ...,8/18/2016,No,XXXX / XXXX / 16 I called Citibank to open a c...
1,Te-2,4/18/2016,Debt collection,Communication tactics,Company believes it acted appropriately as aut...,4/20/2016,No,I'm struggling financially. I called and I off...
2,Te-3,3/23/2016,Credit reporting,Incorrect information on credit report,NaN,3/23/2016,No,"In XXXX of 2015, an automatic payment was conf..."
3,Te-4,6/26/2017,Student loan,Dealing with your lender or servicer,NaN,6/26/2017,NaN,"I submitted a request to XXXX, which is my cur..."
4,Te-5,5/13/2016,Credit reporting,Incorrect information on credit report,Company has responded to the consumer and the ...,5/13/2016,No,A state tax lien was filed against me XXXX / X...


In [59]:
# df1.dropna(inplace=True)

In [37]:
df1.drop(columns=['Date-received', 'Date-sent-to-company','Complaint-ID'], axis=1, inplace=True)

In [38]:
df1['cleaned_tokens'] = df1['Consumer-complaint-summary'].apply(lambda x: tokenize_and_stem(x))
df1['word_count'] = df1['cleaned_tokens'].apply(lambda x: len(x))

In [39]:
df1.drop(columns=['Consumer-complaint-summary','word_count', 'Company-response', 'Consumer-disputes'], axis=1, inplace=True)

In [40]:
le2 = preprocessing.LabelEncoder()
#df1['Company-response'] = le2.fit_transform(df1['Company-response'])
#df1['Consumer-disputes'] = le2.fit_transform(df1['Consumer-disputes'])
df1['Complaint-reason'] = le2.fit_transform(df1['Complaint-reason'])
df1['Transaction-Type'] = le2.fit_transform(df1['Transaction-Type'])

In [41]:
tfidf_vectorizer1 = TfidfVectorizer(max_df=0.5, max_features=200000,
                                 min_df=0.1, stop_words='english',
                                 use_idf=True, ngram_range=(1,3))

%time tfidf_matrix1 = tfidf_vectorizer1.fit_transform(df1['cleaned_tokens'])


print(tfidf_matrix1.shape)

Wall time: 20.8 s
(18543, 61)


In [42]:
tfidf_df1 = pd.DataFrame(tfidf_matrix1.A, columns=tfidf_vectorizer1.get_feature_names(),index=df1.index)
#tfidf_df1 = tfidf_df1.drop(columns=['es', 'make'], axis=1)

In [43]:
a = set(tfidf_df1.columns)

In [44]:
b = set(tfidf_vectorizer.get_feature_names())

In [45]:
if a == b:
    print('true')

In [46]:
predict1 = model.predict(tfidf_df1)

In [89]:
for x in tfidf_vectorizer1.get_feature_names(): 
    print(x, ' ', end='')
print('')
for y in tfidf_vectorizer.get_feature_names():
    print(y, ' ', end='')

00  account  ai  asked  bank  called  card  company  credit  credit report  date  days  debt  des  el  en  es  et  help  information  je  la  le  les  letter  loan  los  make  mi  mon  money  mortgage  ne  number  paid  para  pas  pay  payment  payments  phone  por  pour  que  received  report  request  said  se  sent  service  time  told  una  une  xx  xx xx  xx xx xxxx  xx xxxx  xxxx xxxx xxxx  years  
00  account  ai  asked  bank  called  card  company  credit  credit report  date  days  debt  des  el  en  et  help  information  je  la  le  les  letter  loan  los  mi  mon  money  mortgage  ne  number  paid  para  pas  pay  payment  payments  phone  por  pour  que  received  report  request  said  se  sent  service  time  told  una  une  xx  xx xx  xx xx xxxx  xx xxxx  xxxx xxxx xxxx  years  

In [48]:
df1 = pd.read_csv('test.csv')
df1.drop(columns=['Date-received', 'Transaction-Type', 'Complaint-reason', 'Company-response', 'Date-sent-to-company', 'Consumer-disputes', 'Consumer-complaint-summary'], axis=1, inplace=True)
df1['Complaint-status'] = predict1
df1.head()

,Complaint-ID,Complaint-status
0,Te-1,Closed with explanation
1,Te-2,Closed with explanation
2,Te-3,Closed with explanation
3,Te-4,Closed with explanation
4,Te-5,Closed with explanation


In [54]:
dir(df1.groupby(['Complaint-status']))

['__bytes__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_accessors',
 '_add_numeric_operations',
 '_agg_doc',
 '_aggregate',
 '_aggregate_generic',
 '_aggregate_item_by_item',
 '_aggregate_multiple_funcs',
 '_apply_filter',
 '_apply_to_column_groupbys',
 '_apply_whitelist',
 '_assure_grouper',
 '_block_agg_axis',
 '_bool_agg',
 '_builtin_table',
 '_choose_path',
 '_concat_objects',
 '_constructor',
 '_cumcount_array',
 '_cython_agg_blocks',
 '_cython_agg_general',
 '_cython_table',
 '_cython_transform',
 '_decide_output_index',
 '_def_str',
 '_define_paths',
 '_deprecations',
 '_dir_additio

In [55]:
df1.groupby(['Complaint-status']).count()

,Complaint-ID
Complaint-status,
Closed,816
Closed with explanation,15353
Closed with monetary relief,940
Closed with non-monetary relief,1432
Untimely response,2
